<a href="https://colab.research.google.com/github/robfalck/dymos_tutorial/blob/main/02_dymos_simple_solver_boundary_value_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dymos: Posing the cannonball boundary value problem as a residuals problem.

Sometimes the need to invoke an optimizer can be a hinderance.
In this simple cannonball example, we showed that the problem is not truly an optimization problem.
The number of equality constraints is equal to the number of design variables, so we should be able to solve the problem with a nonlinear solver like a Newton solver.

We will accomplish this by making the duration of the phase a solver-driven variable, and its associated residual will be the final altitude of the cannonball.


In [1]:
!pip install dymos

In [2]:
import numpy as np
import openmdao.api as om
import dymos as dm

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In [3]:
class ProjectileODE(om.ExplicitComponent):

  def initialize(self):
    self.options.declare('num_nodes', types=int,
                         desc='the number of points at which this ODE is simultaneously evaluated')

  def setup(self):
    nn = self.options['num_nodes']
    self.add_input('vx', shape=(nn,), units='m/s')
    self.add_input('vy', shape=(nn,), units='m/s')

    self.add_output('x_dot', shape=(nn,), units='m/s',
                    tags=['state_rate_source:x', 'state_units:m'])

    self.add_output('y_dot', shape=(nn,), units='m/s',
                    tags=['state_rate_source:y', 'state_units:m'])

    self.add_output('vx_dot', shape=(nn,), units='m/s**2',
                    tags=['state_rate_source:vx', 'state_units:m/s'])

    self.add_output('vy_dot', shape=(nn,), units='m/s**2',
                    tags=['state_rate_source:vy', 'state_units:m/s'])
    
    self.declare_partials(of='*', wrt='*', method='fd')
    
  def compute(self, inputs, outputs):
    outputs['x_dot'][:] = inputs['vx']
    outputs['y_dot'][:] = inputs['vy']
    outputs['vx_dot'][:] = 0.0
    outputs['vy_dot'][:] = -9.81
    


# Using a solver to converge the defects

Since we'll be running this example without a driver, we need to treat the defect _constraints_ as _residuals_ be be handled by a nonlinear solver.

In Dymos, this is done using the option `solve_segments`. 
This option can be applied to each state in the `set_state_options` method, or to the transcription.  When provided to the transcription, it becomes the default for every state within the phase.

To use `solve_segments`, it takes values of either `'forward'` or `'backward'`.  When we use `solve_segments`, to ensure that the number of variables equals the number of residuals, we have to `fix` the value of the state at one end of the phase.

When we use `solve_segments='forward'`, the solver is not allowed to vary the initial value in the phase, but is allowed to vary the rest.  In effect, this results in a forward propagation of the state.

Similarly, when we use `solve_segments='backward'`, the final value of the state in the phase is treated as fixed by the solver.

* The state options `fix_initial` and `fix_final` apply to whether the
  optimizer is allowed to change those values.

* The value of `solve_segments` determine which values are allowed to be varied by the solver.

* When using a shooting method, the optimizer may vary the initial state value, for instance, and then the solver can propagate forward from the using `solve_segments='forward'`, for instance.

* Attempting to use an invalid combination, such as `fix_final=True` and `solve_segments='forward'` will result in an error.




In [4]:
prob = om.Problem()

traj = prob.model.add_subsystem('traj', dm.Trajectory())

phase = traj.add_phase('phase',
                       dm.Phase(ode_class=ProjectileODE,
                                transcription=dm.Radau(num_segments=1, order=3, solve_segments=True)))

phase.set_time_options(fix_initial=True, duration_bounds=(1, 1000), units='s')

phase.set_state_options('x', fix_initial=True)
phase.set_state_options('y', fix_initial=True)
phase.set_state_options('vx', fix_initial=True)
phase.set_state_options('vy', fix_initial=True)

# Setup the problem

In [5]:
prob.setup()

# Provide initial guesses

In [6]:
prob.set_val('traj.phase.t_initial', 0.0, units='s')
prob.set_val('traj.phase.t_duration', 15.0, units='s')

prob.set_val('traj.phase.states:x', phase.interpolate(ys=[0, 100], nodes='state_input'), units='m')
prob.set_val('traj.phase.states:y', phase.interpolate(ys=[0, 0], nodes='state_input'), units='m')
prob.set_val('traj.phase.states:vx', phase.interpolate(ys=[100, 100], nodes='state_input'), units='m/s')
prob.set_val('traj.phase.states:vy', phase.interpolate(ys=[100, -100], nodes='state_input'), units='m/s')

# Propagating the trajectory using solve_segments

We can now simply run the model, using either the standard OpenMDAO `Problem.run_model` method, or by using `dymos.run_problem(run_driver=False)`

In [7]:
dm.run_problem(prob, run_driver=False, simulate=True, make_plots=True)


Simulating trajectory traj
Done simulating trajectory traj


In [8]:
# TODO: Make these more automatic (and use an interactive plot library like bokeh)

from IPython.display import display
from ipywidgets import widgets, GridBox

items = [widgets.Image(value=open(f'plots/states_{state}.png', 'rb').read()) for state in ['x', 'y', 'vx', 'vy']] + \
[widgets.Image(value=open(f'plots/state_rates_{state}.png', 'rb').read()) for state in ['x', 'y', 'vx', 'vy']]
widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 500px)"))


GridBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01 \x08\x06\x00\x…

The result is a propagated discrete trajectory without the use of an optimizer.  But the solver is not yet controlling the duration of the phase, so this is still the equivalent of the initial value problem.  In the next section, we'll set up the machinery that allows that to happen.

# So what's the difference between `simulate` and `solve_segments`?

When we invoke `solve_segments`, the propagation is achieved by the iteration of an OpenMDAO system.  As a result, we can take the derivatives _across_ that propagation, and use this propagation in optimization problems.

Using `simulate` does not currently provide the derivatives of the final state with respect to the initial, so that method is useful for generating guesses or verifying results, but not for use within an actual optimization.

# Letting the time duration be controlled by a solver.

In order to allow the time duration to be controlled by a solver, we're going to make a few changes to our problem structure.

First, we're going to add an OpenMDAO `BalanceComp` to our system, after the trajectory.  This BalanceComp will provide a value for `t_duration`.

We will also change the time options for the phase such that `t_duration` is allowed to be an input (using option `input_duration=True`).

The BalanceComp works by varying its implicit output variable (`t_duration`) until the residual is zero.
The residual is calculated as the difference between an associated 'left hand side' (lhs) and 'right hand side' (rhs).
In this case, the lhs will be the output (since its value is changed by changing `t_duration`, and we will leave the rhs unconnected (it will default to zero).

In [9]:
prob = om.Problem()

traj = prob.model.add_subsystem('traj', dm.Trajectory())

bal = prob.model.add_subsystem('bal',
                               om.BalanceComp('t_duration', units='s', eq_units='m'))

phase = traj.add_phase('phase',
                       dm.Phase(ode_class=ProjectileODE,
                                transcription=dm.Radau(num_segments=1, order=3, solve_segments=True)))

phase.set_time_options(fix_initial=True, input_duration=True, units='s')

phase.set_state_options('x', fix_initial=True)
phase.set_state_options('y', fix_initial=True)
phase.set_state_options('vx', fix_initial=True)
phase.set_state_options('vy', fix_initial=True)

prob.model.connect('bal.t_duration', 'traj.phase.t_duration')
prob.model.connect('traj.phase.timeseries.states:y', 'bal.lhs:t_duration', src_indices=[-1])

# Adding a nonlinear solver and a linear solver

When we use `solve_segments`, Dymos automatically adds linear and nonlinear solvers to the Phase to converge the dynamics.

In this case, we're also introducing implicit behavior at the top level of our model (the implicit BalanceComp is a sibling component of the Trajectory).

So now we'll add the necessary solvers to the top of the system.

The linear solver here is needed in order to assemble the derivatives across our iterative system.

In [10]:
prob.model.linear_solver = om.DirectSolver()
prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=True)

# Setup the problem

In [11]:
prob.setup()

# Provide initial guesses

In [12]:
prob.set_val('traj.phase.t_initial', 0.0, units='s')
prob.set_val('traj.phase.t_duration', 15.0, units='s')

prob.set_val('traj.phase.states:x', phase.interpolate(ys=[0, 100], nodes='state_input'), units='m')
prob.set_val('traj.phase.states:y', phase.interpolate(ys=[0, 0], nodes='state_input'), units='m')
prob.set_val('traj.phase.states:vx', phase.interpolate(ys=[100, 100], nodes='state_input'), units='m/s')
prob.set_val('traj.phase.states:vy', phase.interpolate(ys=[100, -100], nodes='state_input'), units='m/s')

# Run the problem

Here we could either use `om.run_model()` or `dm.run_problem(prob, run_driver=False)`

In [13]:
dm.run_problem(prob, run_driver=False, simulate=True, make_plots=True)

NL: Newton Converged in 5 iterations

Simulating trajectory traj
Done simulating trajectory traj


In [14]:
# TODO: Make these more automatic (and use an interactive plot library like bokeh)

from IPython.display import display
from ipywidgets import widgets, GridBox

items = [widgets.Image(value=open(f'plots/states_{state}.png', 'rb').read()) for state in ['x', 'y', 'vx', 'vy']] + \
[widgets.Image(value=open(f'plots/state_rates_{state}.png', 'rb').read()) for state in ['x', 'y', 'vx', 'vy']]
widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 500px)"))

GridBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01 \x08\x06\x00\x…

# Summary

Once again we've solved the boundary value problem.
This time, no optimizer was used to converge the state dynamics.